In [1]:
from mlcloud import ferranti_exec, ferranti_print_logs

def ferranti_exec_cpu(command):
    sbatch_command = """sbatch <<'EOF'
#!/bin/bash
#SBATCH --time=3-00:00:00  # Runtime in D-HH:MM:SS    
#SBATCH --output=/weka/luxburg/sbordt10/logs/exec_cpu/%j.out  
#SBATCH --error=/weka/luxburg/sbordt10/logs/exec_cpu/%j.err   
#SBATCH --open-mode=append
#SBATCH --job-name=ferranti-exec-cpu
#SBATCH --partition=cpu-ferranti
#SBATCH --nodes=1  
#SBATCH --ntasks=1       
#SBATCH --cpus-per-task=32    
#SBATCH --mem=256G   

scontrol show job ${SLURM_JOB_ID}
export WANDB__SERVICE_WAIT=6000
source activate tp-theory-new
"""
    ferranti_exec(sbatch_command + command + "\nEOF")


def ferranti_exec_H100(command):
    sbatch_command = """sbatch <<'EOF'
#!/bin/bash
#SBATCH --time=3-00:00:00  # Runtime in D-HH:MM:SS    
#SBATCH --output=/weka/luxburg/sbordt10/logs/exec_h100/%j.out  
#SBATCH --error=/weka/luxburg/sbordt10/logs/exec_h100/%j.err   
#SBATCH --open-mode=append
#SBATCH --job-name=ferranti-exec-h100
#SBATCH --partition=h100-ferranti
#SBATCH --nodes=1  
#SBATCH --ntasks=1       
#SBATCH --cpus-per-task=8    
#SBATCH --mem=128G   
#SBATCH --gres=gpu:1 

scontrol show job ${SLURM_JOB_ID}
nvidia-smi
export NCCL_TIMEOUT=1800000
export WANDB__SERVICE_WAIT=6000
source activate tp-theory-new
"""
    ferranti_exec(sbatch_command + command + "\nEOF")

## Monitor and control jobs

In [4]:
ferranti_exec("cd OLMo && git pull")

Updating ca73eca0..094d0e99
Fast-forward
 .../notebooks/contorl_ferranti.ipynb               | 1512 ++++++++++++++++++++
 single-training-run/notebooks/mlcloud.py           |  104 ++
 single-training-run/scripts/OLMo-1B.yaml           |    2 +-
 3 files changed, 1617 insertions(+), 1 deletion(-)
 create mode 100644 single-training-run/notebooks/contorl_ferranti.ipynb
 create mode 100644 single-training-run/notebooks/mlcloud.py

From https://github.com/sbordt/OLMo
   ca73eca0..094d0e99  single-training-run -> origin/single-training-run



In [7]:
ferranti_exec("squeue --me")

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
             53071 h100-ferr     olmo sbordt10 PD       0:00      1 (Resources)




In [5]:
ferranti_exec("cd OLMo/single-training-run/scripts && sbatch train_4xH100.sh")

Submitted batch job 53071




In [11]:
# cancel a specific job
ferranti_exec("scancel 45262")

In [6]:
ferranti_exec("squeue --me")

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
             53071 h100-ferr     olmo sbordt10 PD       0:00      1 (Resources)




In [8]:
ferranti_print_logs("/weka/luxburg/sbordt10/logs/single-training-run", num_files=2)

Content of /weka/luxburg/sbordt10/logs/single-training-run/53071.err:
W0428 18:47:19.659000 23456247972352 torch/distributed/run.py:757] 
W0428 18:47:19.659000 23456247972352 torch/distributed/run.py:757] *****************************************
W0428 18:47:19.659000 23456247972352 torch/distributed/run.py:757] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0428 18:47:19.659000 23456247972352 torch/distributed/run.py:757] *****************************************
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sbordt. Use `wandb login --relogin` to force relogin
wandb: ERROR failed to upsert bucket: returned error 403 Forbidden: {"errors":[{"message":"permission denied","path":["upsertBucket"],"extensions":{"code":"PERMISSION_ERROR"